# Practica 2 -  Clasificación supervisada en scikit-learn

## Mineria de Datos 2017/2018

* **Hernan Indibil de La Cruz Calvo**
* **Alejandro Martin Simon Sanchez**

## Indice
1. Introduccion
2. Clasificadores y metodos de evaluacion


## 1. Introduccion
Esta práctica tendrá dos partes:

Primero estudiaremos la API de algunos de los clasificadores más utilizados en `scikit-learn` para conocer los distintos hiperparámetros que los configuran y estudiar los modelos resultantes.

Segundo estudiaremos métodos de selección de modelos, orientados a obtener una configuración óptima de los hiperparámetros para nuestros clasificadores.

In [2]:
# Always load all scipy stack packages
import numpy as np
import pandas as pd
from scipy import stats, integrate
import matplotlib as mpl
import matplotlib.pyplot as plt

import seaborn as sns
sns.set(color_codes=True)

In [3]:
# This code configures matplotlib for proper rendering
%matplotlib inline
mpl.rcParams["figure.figsize"] = "8, 4"
import warnings
warnings.simplefilter("ignore")

In [4]:
# Se establece una semilla predeterminada para que los experimentos sean reproducibles
seed = 6470
np.random.seed(seed)

* **Lo siguiente es cargar los datos que se van a utilizar.**

    Se usa como label la variable categórica.

In [5]:
# Diccionario de nombre: fichero, con los datos de los dataframe a cargar
files = {
    'pima': '../data/pima.csv',
    'wisconsin': '../data/wisconsin.csv'
}

In [6]:
# Se cargan los dataframes
dfs = {name: pd.read_csv(file, dtype={ "label": 'category'}) for name, file in files.items()}

Como vimos en la práctica anterior, las variables de Pima "plas", "pres", "skin", "insu" y "mass" tienen los valores perdidos codificados como 0, ya que es imposible que una persona viva tenga valor 0 en cualquiera de ellas.
Podemos cambiar los 0 por NaN en el dataframe original sin perder información ni sobreajustar de ninguna forma, ya que no usamos información del conjunto de datos.

In [7]:
dfs['pima']['plas'] = dfs['pima']['plas'].replace(0, np.nan)
dfs['pima']['pres'] = dfs['pima']['pres'].replace(0, np.nan)
dfs['pima']['skin'] = dfs['pima']['skin'].replace(0, np.nan)
dfs['pima']['insu'] = dfs['pima']['insu'].replace(0, np.nan)
dfs['pima']['mass'] = dfs['pima']['mass'].replace(0, np.nan)

Para el dataframe Wisconsin la variable Patient ID no aporta nada beneficioso al proceso de clasificación, sólo sobreajuste. No tiene nada que ver con la variable clase. Por ello, procedemos a eliminarla.

In [8]:
dfs['wisconsin'] = dfs['wisconsin'].drop('patientId', 1)

Ahora procedemos a actualizar el diccionario de dataframes para que tenga la siguiente estructura:

* dfs
    * pima
        * train
            * atts
            * label
        * test
            * atts
            * label
    * wisconsin
        * train
            * atts
            * label
        * test
            * atts
            * label

La función utilizada no solo crea la estructura para los dataframes pima y wisconsin, sino para todos los dataframes que haya en el diccionario que se le pase. Realiza el proceso de holdout también para todos los dataframes.

In [17]:
from sklearn.model_selection import train_test_split

def holdout(dframe, seed, tsize = 0.2):
    dfAttributes = dframe.drop('label', 1)
    dfLabel = dframe['label']

    df = {}
    df['train'] = {}
    df['test'] = {}

    df['train']['atts'], df['test']['atts'], df['train']['label'], df['test']['label'] = train_test_split(
        dfAttributes,
        dfLabel,
        test_size = tsize,
        random_state = seed,
        stratify = dfLabel)

    return df

dfsh = { name: holdout(dframe, seed, 0.2) for name, dframe in dfs.items() }

## 2. Selección de modelos

### 2.1 Uso del algoritmo Grid Search

En este apartado se utiliza el algoritmo Grid Search para encontrar la configuración óptima para los distintos estimadores. Dicho algoritmo recibe como parámetros el estimador a configurar y los  a ajustar con una lista de los posibles valores que puedan tomar.
Por ello, para poder utilizarlo debemos primero ver cómo se crean los estimadores y qué variables pueden tener y en qué rango deben ser ajustadas.

#### 2.1.1 Tratamiento de valores perdidos

Sobre los distintos dataframes es posible realizar un tratamiento de los valores perdidos.
Es posible realizarlo mediante un Imputer de scikit, que realizará automáticamente el cambio de los mismos por la media, mediana o moda según cómo se indique la estrategia.

In [33]:
from sklearn.preprocessing import Imputer

In [37]:
# Ejemplo

# El siguiente imputer sustituye los np.nan por la media (Valores por defecto)
# Primero definimos el modelo
imp = Imputer()

# Ahora lo entrenamos
imp = imp.fit(dfsh['pima']['train']['atts'])

# Finalmente lo podemos usar para transformar el dataframe de la siguiente forma
X = imp.transform(dfsh['pima']['train']['atts'])
Y = imp.transform(dfsh['pima']['test']['atts'])

# El resultado es una matriz, por lo que debemos transformarla de nuevo en un dataframe
train_attsFull = pd.DataFrame(X, columns = dfsh['pima']['train']['atts'].columns)
test_attsFull = pd.DataFrame(Y, columns = dfsh['pima']['test']['atts'].columns)

train_attsFull.describe()

,preg,plas,pres,skin,insu,mass,pedi,age
count,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000,614.000000
mean,3.884365,122.502463,72.519591,29.392111,156.279874,32.596053,0.480606,33.285016
std,3.396762,30.833718,11.955500,8.914926,87.090249,7.009831,0.331490,11.698435
min,0.000000,44.000000,24.000000,7.000000,15.000000,18.200000,0.078000,21.000000
25%,1.000000,100.000000,64.000000,25.250000,120.500000,27.500000,0.248000,24.000000
50%,3.000000,119.000000,72.519591,29.392111,156.279874,32.400000,0.384000,29.000000
75%,6.000000,142.000000,80.000000,33.000000,156.279874,36.875000,0.639250,40.000000
max,17.000000,198.000000,114.000000,99.000000,846.000000,67.100000,2.329000,81.000000


De esta forma tenemos que por parte del Imputer las variables a configurar son:
* missing_values: variable con el valor con el que se codifican los valores perdidos. Por defecto missing_values = 'NaN', que sustituye los np.nan.
* strategy: variable que indica con qué se reemplazan los valores perdidos. Puede ser: 'mean', 'median' y 'most_frequent'. Por defecto strategy = 'mean'.
* axis: eje en el que se hace la imputación (0 para columnas, 1 para filas). Por defecto axis = 0.

La información ha sido obtenida de la documentación encontrada en [sklearn.preprocessing.Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html#sklearn.preprocessing.Imputer.fit_transform).

Una vez terminado el tratamiento de valores perdidos, podemos proceder a crear los clasificadores. En esta práctica se utilizarán dos métodos de clasificación: KNN y árbol de decisión.

#### 2.1.2 Clasificador KNN

KNN es un clasificador sencillo de entender y configurar, ya que solo tendremos que fijar el parámetro `k` que determina el número de vecinos con los que compararemos.

Se trata de un algoritmo perezoso, es decir, que no realiza fase de aprendizaje previa porque computa los parámetros necesarios para la clasificación durante el propio proceso de clasificación. Aunque esto pueda parecer una ventaja puede llegar a resultar ineficiente para bases de datos con muchas instancias. Además, es muy sensible a cambios en los datos de training.

In [31]:
from sklearn import neighbors

In [41]:
# Ejemplo

# Se prueba el clasificador con un valor de k = 5 vecinos sin pesar por la distancia (Por defecto).
# La distancia usada es la de Minkowski
model = neighbors.KNeighborsClassifier()
# Como es perezoso solo se inicializa su estado.

# A continuacion se aprenden los datos del conjunto de training.
knn = model.fit(train_attsFull, dfsh['pima']['train']['label'])

predictionKNN = knn.predict(test_attsFull)

De esta forma tenemos que por parte del clasificador KNN los hiperparámetros a configurar son:
* n_neighbors: variable con el número de vecinos usados. Por defecto n_neighbors = 5. Es la K de KNN.
* weights: variable que indica cómo pesar los vecinos a la hora de clasificar, pudiendo elegir entre que todos pesen lo mismo ('uniform') o pesar por la inversa de la distancia ('distance'). También es posible pasar por parámetro una función que devuelva el peso recibiendo como argumento un array de distancias. Por defecto weights = 'uniform'.
* metric: métrica de distancia a utilizar. Las posibles métricas pueden encontrarse en [class DistanceMetric](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.DistanceMetric.html). Por defecto metric = 'minkowski'.
* p: parámetro para la métrica de distancia 'minkowsky', donde la distancia entre x e y se calcula como sum(|x - y|^p)^(1/p). Por ejemplo para p = 1 es la distancia de Manhattan y con p = 2 es la Euclídea. Por defecto p = 2.
* metric_params: otros argumentos que pueden ser usados en la función de distancia escogida. Por ejemplo en la distancia de Mahalanobis o en la de WMinkowsky.

Hay más variables que sirven para mejorar la eficiencia pero que no afectan al resultado de la clasificación, que afectan por ejemplo al nivel de concurrencia. Por ello, no las trataremos de momento.

La información ha sido obtenida de la documentación encontrada en [sklearn.neighbors.KNeighborsClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html#sklearn.neighbors.KNeighborsClassifier).

#### 2.1.3 Árboles de decisión

En la práctica anterior ya se trabajó con árboles de decisión, por lo que ahora nos centraremos más en los hiperparámetros:

In [42]:
from sklearn import tree

In [44]:
# Ejemplo

# Iniciamos el modelo usando la impureza Gini como criterio para las divisiones, sin profundidad máxima.
model = tree.DecisionTreeClassifier(random_state = seed)

# Entrenamos el modelo
classifier = model.fit(train_attsFull, dfsh['pima']['train']['label'])

# Obtenemos la predicción
prediction = classifier.predict(test_attsFull)

De esta forma tenemos que por parte del árbol de decisión los hiperparámetros a configurar son:
* criterion: variable con el criterio para hacer las divisiones. Puede ser 'gini' para usar la impureza Gini o 'entropy' para usar la ganancia de información. Por defecto criterion = 'gini'.
* splitter: variable que permite decidir entre buscar el mejor corte con 'best' o el mejor corte aleatorio con 'random'. Por defecto splitter = 'best'.
* max_depth: variable que establece la profundidad máxima del árbol. Reducirla puede ayudar a evitar el sobreajuste. Por defecto max_depth = None.
* min_samples_split: variable que permite determinar el mínimo número de instancias necesarias para dividir un nodo interno. Aumentar el número permite disminuir el sobreajuste. Por defecto min_samples_split = 2.
* min_samples_leaf: variable que permite determinar el mínimo número de casos necesarios para que un nodo sea hoja. Aumentar el número permite disminuir el sobreajuste. Por defecto min_samples_leaf = 1.
* min_weight_fraction_leaf: se usa cuando a la hora de entrenar se han especificado pesos para los distintos casos. De no especificarse todos los casos tendrán el mismo peso. La variable indica la minima fracción de peso del total de pesos necesaria para que un nodo pueda ser hoja. Por defecto min_weight_fraction_leaf = 0. (debe ser float).
* max_features: el número de variables a considerar a la hora de buscar el mejor corte. Se puede especificar un entero, un flotante con el porcentaje, 'auto' o 'sqrt' para tomar la raíz del número total de variables (¿por qué auto hace siempre lo mismo?), 'log2' para tomar el logaritmo en base 2 o None para tomar max_features = numero total de variables. Por defecto max_features = None.
* max_leaf_nodes: máximo número de nodos hoja, donde el árbol se construirá con primero-mejor pesando con la impureza. Por defecto max_leaf_nodes = None.
* min_impurity_decrease: un nodo se divide si al hacerlo se produce una disminución de impureza mayor o igual a éste valor. Aumentar el valor disminuye el sobreajuste. Por defecto min_impurity_decrease = 0.
* class_weight: indica el peso de cada etiqueta posible de la variable clase (también puede usarse en problemas multiclase). Puede pasarse un diccionario etiqueta: peso, una lista de diccionarios, 'balanced' para que pese las etiquetas en función de la inversa de sus frecuencias de aparición o None para que todas tengan el mismo peso. Por defecto class_weight = None.

Hay más hiperparámetros que sirven para mejorar la eficiencia pero que no afectan al resultado de la clasificación, como presort. También tenemos el hiperparámetro random_state, para introducir la semilla. Por ello, no las trataremos de momento.

La información ha sido obtenida de la documentación encontrada en [sklearn.tree.DecisionTreeClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeClassifier.html)

In [ ]:
from sklearn.preprocessing import Imputer

In [ ]:
imp = Imputer(missing_values = '0', strategy = 'mean', axis = 0)
imp.fit(dfs[pima])

In [ ]:
from sklearn.model_selection import GridSearchCV